随机森林

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy
import pandas 
import scipy 
import seaborn 
import sklearn.metrics
import warnings

In [12]:
data = pandas.read_excel('第二问数据.xlsx')
data = data.drop('Unnamed: 0',axis=1)
data.head()

,ldeltotal,mpaccdigit,faq,mmse,adas13,pteducat,age,adas11,ravlt_immediate,ravlt_perc_forgetting,adasq4,ravlt_learning,ravlt_forgetting,dx_id
0,10,-4.316700,0,28,18.67,16,74.300003,10.67,44,54.545502,5,4,6,0
1,2,-16.290501,10,20,31.00,18,81.300003,22.00,22,100.000000,8,1,4,4
2,4,-8.433050,0,27,21.33,10,67.500000,14.33,37,36.363602,6,7,4,3
3,12,-1.890180,0,29,14.67,16,73.699997,8.67,37,44.444401,4,4,4,0
4,3,-10.163000,0,25,25.67,13,80.400002,18.67,30,83.333298,7,1,5,3


In [13]:
X = data.drop('dx_id',axis=1)
Y = data['dx_id']
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X, 
                                            Y,
                                            test_size=0.5,
                                            random_state=0)

In [64]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
            max_depth=7, max_features=7, 
            n_estimators=48,
            min_samples_leaf=1,
            oob_score=False,
           )
rfc.fit(Xtrain, Ytrain)
Ypred = rfc.predict(Xtest)
from sklearn.metrics import classification_report
result1 = classification_report(Ytest, Ypred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(Ytest,Ypred)
print("Accuracy:",result2)

Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.71      0.65       256
           1       0.44      0.22      0.29       174
           2       0.68      0.77      0.72       206
           3       0.87      0.93      0.90       357
           4       0.88      0.83      0.86       220

   micro avg       0.74      0.74      0.74      1213
   macro avg       0.69      0.69      0.68      1213
weighted avg       0.72      0.74      0.72      1213

Accuracy: 0.7361912613355317


In [15]:
#交叉验证
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
import numpy as np
param_dist = {
        'n_estimators':range(50,60),
        'max_depth':range(1,12),
        'max_features':np.arange(1,8),
        'min_samples_leaf':np.arange(1,12),
        'min_samples_split':np.arange(2, 2+20, 1),
        'criterion':['gini', 'entropy']
        }
cvv = KFold(5)
grid = RandomizedSearchCV(rfc,param_dist,cv = cvv,n_iter=100,refit="roc_auc",n_jobs = -1)
  
#在训练集上训练
grid.fit(Xtrain,Ytrain)
#返回最优的训练器
best_estimator = grid.best_estimator_
print(best_estimator)
#输出最优训练器的精度
print(grid.best_score_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=7, max_features=7, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=9, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=53, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
0.7079207920792079


In [56]:
#最佳生成数量：网格法搜索超参
from sklearn.model_selection import GridSearchCV
def random_forest_predict(Xtrain, Ytrain, Xtest, Ytest):
    param = {'n_estimators':range(45,55), 'max_depth':range(2,8),'max_features':numpy.arange(5,14)}#在一定范围内设置遍历集
    gsearch = GridSearchCV(estimator=RandomForestClassifier(min_samples_leaf=1,
                                                            oob_score=False,
                                                            random_state=42),
                            param_grid=param, 
                            iid=False, cv=10, n_jobs=-1)
    gsearch.fit(Xtrain, Ytrain)
    print(gsearch.best_params_, gsearch.best_score_)
random_forest_predict(Xtrain, Ytrain, Xtest, Ytest)
#设置K折：2、5、10、264后

{'max_depth': 4, 'max_features': 11, 'n_estimators': 48} 0.7220175234533823


Xgboost

In [72]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

#分类器使用 xgboost
from xgboost import XGBClassifier
model = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
       colsample_bylevel=1, colsample_bynode=1,
       colsample_bytree=0.8733333333333333, early_stopping_rounds=None,
       enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
       grow_policy='depthwise', importance_type=None,
       interaction_constraints='', learning_rate=0.01, max_bin=256,
       max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
       min_child_weight=2, monotone_constraints='()',
       n_estimators=116, n_jobs=0, num_parallel_tree=1,
       objective='multi:softprob', predictor='auto', random_state=0,
       reg_alpha=0, reg_lambda=1, sampling_method='uniform',
       scale_pos_weight=None, subsample=0.731578947368421,
       tree_method='exact', use_label_encoder=False, validate_parameters=1,
       verbosity=None)
model.fit(Xtrain, Ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
       colsample_bylevel=1, colsample_bynode=1,
       colsample_bytree=0.8733333333333333, early_stopping_rounds=None,
       enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
       grow_policy='depthwise', importance_type=None,
       interaction_constraints='', learning_rate=0.01, max_bin=256,
       max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
       min_child_weight=2, missing=nan, monotone_constraints='()',
       n_estimators=116, n_jobs=0, num_parallel_tree=1,
       objective='multi:softprob', predictor='auto', random_state=0,
       reg_alpha=0, reg_lambda=1, sampling_method='uniform',
       scale_pos_weight=None, subsample=0.731578947368421,
       tree_method='exact', use_label_encoder=False, validate_parameters=1,
       verbosity=None)

In [73]:
y_pred2 = model.predict(Xtest)

In [74]:
from sklearn.model_selection import KFold # k折交叉验证，默认5
import numpy as np
param_dist = {
        'n_estimators':range(80,200,4),
        'max_depth':range(2,15,1),
        'learning_rate':np.linspace(0.01,2,20),
        'subsample':np.linspace(0.7,0.9,20),
        'colsample_bytree':np.linspace(0.5,0.98,10),
        'min_child_weight':range(1,9,1)
        }

cvv = KFold(5)
grid = RandomizedSearchCV(model,param_dist,cv = cvv,n_iter=100,refit="roc_auc",n_jobs = -1)
  
#在训练集上训练
grid.fit(Xtrain,Ytrain)
#返回最优的训练器
best_estimator = grid.best_estimator_
print(best_estimator)
#输出最优训练器的精度
print(grid.best_score_)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
       colsample_bylevel=1, colsample_bynode=1,
       colsample_bytree=0.6599999999999999, early_stopping_rounds=None,
       enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
       grow_policy='depthwise', importance_type=None,
       interaction_constraints='', learning_rate=0.11473684210526315,
       max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=2,
       max_leaves=0, min_child_weight=7, missing=nan,
       monotone_constraints='()', n_estimators=84, n_jobs=0,
       num_parallel_tree=1, objective='multi:softprob', predictor='auto',
       random_state=0, reg_alpha=0, reg_lambda=1,
       sampling_method='uniform', scale_pos_weight=None,
       subsample=0.8263157894736842, tree_method='exact',
       use_label_encoder=False, validate_parameters=1, verbosity=None)
0.7054455445544554


C:\Users\Kal\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [75]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
res = classification_report(Ytest, y_pred2)
print("Classification Report:",)
print (res)
res2 = accuracy_score(Ytest,y_pred2)
print("Accuracy:",res2)

Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.73      0.65       256
           1       0.42      0.16      0.23       174
           2       0.70      0.81      0.75       206
           3       0.86      0.91      0.88       357
           4       0.86      0.82      0.84       220

   micro avg       0.73      0.73      0.73      1213
   macro avg       0.68      0.69      0.67      1213
weighted avg       0.71      0.73      0.71      1213

Accuracy: 0.731244847485573
